# 人物検索と動作検索を分けて評価する
TRECVID で FastReID の評価と SlowFast Networksの評価を分けて行うための下準備．

In [74]:
import os
import csv
import cv2
import xml.etree.ElementTree as ET

In [24]:
gallery_shots_dict = {}
with open("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/ins.search.qrels.tv19", "r") as f:
    lines = f.readlines()
    for line in lines:
        query_num, _, shot, _ = line.split()
        if query_num in gallery_shots_dict:
            gallery_shots_dict[query_num].append(shot)
        else:
            gallery_shots_dict[query_num] = [shot]

In [30]:
# 人物の番号
with open("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/query/person_labels.csv", "r") as f:
    rows = csv.reader(f, delimiter=" ")
    person_num_dict = {r[0].capitalize(): r[1] for r in rows}
    
# 動作の番号
with open("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/query/action_labels.csv", "r") as f:
    rows = csv.reader(f, delimiter=" ")
    action_num_dict = {r[0]: r[1] for r in rows}

In [37]:
tree = ET.parse("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/topics/ins.auto.topics.2019.xml")
root = tree.getroot()
# クエリ番号をキーとした辞書
query_num_dict = {child.attrib["num"]: child.attrib for child in root}
# 人物をキーとした辞書
query_person_dict = {}
for child in root:
    if child.attrib["person"] in query_person_dict:
        query_person_dict[child.attrib["person"]].append(child.attrib)
    else:
        query_person_dict[child.attrib["person"]] = [child.attrib]
# 動作をキーとした辞書
query_action_dict = {}
for child in root:
    if child.attrib["action"] in query_action_dict:
        query_action_dict[child.attrib["action"]].append(child.attrib)
    else:
        query_action_dict[child.attrib["action"]] = [child.attrib]

In [66]:
query_person_dict

{'Max': [{'person': 'Max', 'action': 'holding_glass', 'num': '9249'},
  {'person': 'Max', 'action': 'shouting', 'num': '9261'},
  {'person': 'Max', 'action': 'crying', 'num': '9265'}],
 'Ian': [{'person': 'Ian', 'action': 'holding_glass', 'num': '9250'},
  {'person': 'Ian', 'action': 'holding_phone', 'num': '9255'},
  {'person': 'Ian', 'action': 'open_door_enter', 'num': '9259'},
  {'person': 'Ian', 'action': 'eating', 'num': '9263'}],
 'Pat': [{'person': 'Pat', 'action': 'holding_glass', 'num': '9251'},
  {'person': 'Pat', 'action': 'sit_on_couch', 'num': '9253'},
  {'person': 'Pat', 'action': 'drinking', 'num': '9258'}],
 'Denise': [{'person': 'Denise', 'action': 'holding_glass', 'num': '9252'},
  {'person': 'Denise', 'action': 'sit_on_couch', 'num': '9254'}],
 'Phil': [{'person': 'Phil', 'action': 'holding_phone', 'num': '9256'},
  {'person': 'Phil', 'action': 'shouting', 'num': '9262'},
  {'person': 'Phil', 'action': 'go_up_down_stairs', 'num': '9268'}],
 'Jane': [{'person': 'Jane'

In [67]:
# クエリ番号によって動画を選別
query_nums = ['9249', '9261', '9265']

shot_list = []
for num in query_nums:
    shot_list += gallery_shots_dict[num]
shot_set = set(shot_list)

person_list = [person_num_dict[query_num_dict[num]['person']] for num in query_nums]

video_list = []
with open("/net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/trecvid_2019/gallery/features.csv", "r") as f:
    rows = csv.reader(f, delimiter=" ")
    for r in rows:
        shot = os.path.basename(r[5]).rsplit('_', 2)[0]
        person_label = r[7]
        if shot in shot_set and person_label in person_list:
            video_list.append(r)

In [70]:
for v in video_list[10000:10100]:
    print(v[7], v[6], v[5])

8 99 /net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot141_1906_id_1526902.mp4
8 99 /net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot141_401_id_1517107.mp4
8 99 /net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot141_401_id_1517106.mp4
8 99 /net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot141_407_id_1517156.mp4
8 99 /net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot141_407_id_1517155.mp4
8 99 /net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot141_41_id_1514905.mp4
8 99 /net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot141_41_id_1514908.mp4
8 99 /net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot141_425_id_1517219.mp4
8 99 /net/per610a/export/das18a/satoh-lab/share/datasets/eastenders/video_detected/shot141_506_id_1517552.mp4
8 99 /net/p

In [72]:
with open("max_features.csv", "w") as f:
    writer = csv.writer(f, delimiter=" ")
    writer.writerows(video_list)

In [73]:
person_num_dict

{'Bradley': '1',
 'Denise': '2',
 'Dot': '3',
 'Heather': '4',
 'Ian': '5',
 'Jack': '6',
 'Jane': '7',
 'Max': '8',
 'Pat': '9',
 'Phil': '10',
 'Sean': '11',
 'Shirley': '12',
 'Stacey': '13'}

In [77]:
for i, v in enumerate(video_list):
    cap = cv2.VideoCapture(v[5])
    if not cap.isOpened():
        break
        
    cap.set(cv2.CAP_PROP_POS_FRAMES, 16)
    ret, frame = cap.read()
    if ret:
        img_path = "./max_images/" + os.path.basename(v[5]).replace(".mp4", ".png")
        cv2.imwrite(img_path, frame)
        print(i, img_path)

0 ./max_images/shot100_1012_id_1105879.png
1 ./max_images/shot100_1012_id_1105878.png
2 ./max_images/shot100_110_id_1100620.png
3 ./max_images/shot100_291_id_1101500.png
4 ./max_images/shot100_291_id_1101501.png
5 ./max_images/shot100_297_id_1101521.png
6 ./max_images/shot100_300_id_1101532.png
7 ./max_images/shot100_383_id_1101969.png
8 ./max_images/shot100_383_id_1101971.png
9 ./max_images/shot100_4_id_1100184.png
10 ./max_images/shot100_4_id_1100186.png
11 ./max_images/shot100_423_id_1102219.png
12 ./max_images/shot100_423_id_1102231.png
13 ./max_images/shot100_427_id_1102262.png
14 ./max_images/shot100_429_id_1102269.png
15 ./max_images/shot100_431_id_1102280.png
16 ./max_images/shot100_435_id_1102293.png
17 ./max_images/shot100_45_id_1100381.png
18 ./max_images/shot100_466_id_1102494.png
19 ./max_images/shot100_51_id_1100415.png
20 ./max_images/shot100_51_id_1100413.png
21 ./max_images/shot100_542_id_1102893.png
22 ./max_images/shot100_550_id_1102904.png
23 ./max_images/shot100_63

188 ./max_images/shot107_709_id_1161082.png
189 ./max_images/shot107_709_id_1161087.png
190 ./max_images/shot107_709_id_1161070.png
191 ./max_images/shot107_710_id_1161109.png
192 ./max_images/shot107_713_id_1161157.png
193 ./max_images/shot107_713_id_1161156.png
194 ./max_images/shot107_758_id_1161368.png
195 ./max_images/shot108_1772_id_1173290.png
196 ./max_images/shot108_1772_id_1173293.png
197 ./max_images/shot108_1772_id_1173313.png
198 ./max_images/shot108_1772_id_1173292.png
199 ./max_images/shot108_1772_id_1173291.png
200 ./max_images/shot109_1003_id_1179901.png
201 ./max_images/shot109_1003_id_1179906.png
202 ./max_images/shot109_1007_id_1179956.png
203 ./max_images/shot109_1007_id_1179961.png
204 ./max_images/shot109_1079_id_1180460.png
205 ./max_images/shot109_1079_id_1180468.png
206 ./max_images/shot109_1093_id_1180551.png
207 ./max_images/shot109_1093_id_1180552.png
208 ./max_images/shot109_1101_id_1180592.png
209 ./max_images/shot109_1123_id_1180687.png
210 ./max_images/

375 ./max_images/shot116_555_id_1253688.png
376 ./max_images/shot116_555_id_1253697.png
377 ./max_images/shot116_60_id_1250915.png
378 ./max_images/shot116_710_id_1254682.png
379 ./max_images/shot116_710_id_1254695.png
380 ./max_images/shot116_710_id_1254719.png
381 ./max_images/shot116_710_id_1254729.png
382 ./max_images/shot116_710_id_1254743.png
383 ./max_images/shot116_740_id_1254932.png
384 ./max_images/shot116_740_id_1254931.png
385 ./max_images/shot116_740_id_1254930.png
386 ./max_images/shot116_740_id_1254929.png
387 ./max_images/shot116_740_id_1254933.png
388 ./max_images/shot116_808_id_1255261.png
389 ./max_images/shot116_808_id_1255265.png
390 ./max_images/shot116_811_id_1255294.png
391 ./max_images/shot116_811_id_1255292.png
392 ./max_images/shot116_811_id_1255310.png
393 ./max_images/shot116_811_id_1255323.png
394 ./max_images/shot116_811_id_1255339.png
395 ./max_images/shot116_811_id_1255347.png
396 ./max_images/shot116_811_id_1255357.png
397 ./max_images/shot116_811_id_1

564 ./max_images/shot126_2456_id_1355514.png
565 ./max_images/shot126_2456_id_1355522.png
566 ./max_images/shot126_2457_id_1355552.png
567 ./max_images/shot126_2457_id_1355588.png
568 ./max_images/shot126_2457_id_1355587.png
569 ./max_images/shot126_2457_id_1355586.png
570 ./max_images/shot126_2458_id_1355598.png
571 ./max_images/shot126_2488_id_1355793.png
572 ./max_images/shot126_2488_id_1355792.png
573 ./max_images/shot126_2525_id_1355990.png
574 ./max_images/shot126_2553_id_1356077.png
575 ./max_images/shot126_2571_id_1356253.png
576 ./max_images/shot126_2727_id_1357161.png
577 ./max_images/shot126_2727_id_1357157.png
578 ./max_images/shot126_2727_id_1357159.png
579 ./max_images/shot127_1203_id_1363904.png
580 ./max_images/shot127_1203_id_1363903.png
581 ./max_images/shot127_1211_id_1363961.png
582 ./max_images/shot127_1211_id_1363958.png
583 ./max_images/shot127_1219_id_1364009.png
584 ./max_images/shot127_1219_id_1364008.png
585 ./max_images/shot127_1229_id_1364087.png
586 ./max_

749 ./max_images/shot132_237_id_1433763.png
750 ./max_images/shot132_237_id_1433762.png
751 ./max_images/shot132_237_id_1433791.png
752 ./max_images/shot132_237_id_1433800.png
753 ./max_images/shot132_237_id_1433808.png
754 ./max_images/shot132_237_id_1433745.png
755 ./max_images/shot132_237_id_1433746.png
756 ./max_images/shot132_237_id_1433751.png
757 ./max_images/shot132_276_id_1434041.png
758 ./max_images/shot132_276_id_1434043.png
759 ./max_images/shot132_276_id_1434044.png
760 ./max_images/shot132_276_id_1434035.png
761 ./max_images/shot132_568_id_1435464.png
762 ./max_images/shot132_568_id_1435470.png
763 ./max_images/shot132_568_id_1435468.png
764 ./max_images/shot132_568_id_1435467.png
765 ./max_images/shot132_568_id_1435466.png
766 ./max_images/shot132_568_id_1435465.png
767 ./max_images/shot132_568_id_1435397.png
768 ./max_images/shot132_568_id_1435399.png
769 ./max_images/shot132_568_id_1435411.png
770 ./max_images/shot132_568_id_1435432.png
771 ./max_images/shot132_568_id_

...

12354 ./max_images/shot55_319_id_617930.png
12355 ./max_images/shot55_319_id_617935.png
12356 ./max_images/shot55_340_id_618010.png
12357 ./max_images/shot55_340_id_618008.png
12358 ./max_images/shot5_540_id_59238.png
12359 ./max_images/shot5_540_id_59239.png
12360 ./max_images/shot5_542_id_59249.png
12361 ./max_images/shot5_542_id_59250.png
12362 ./max_images/shot5_546_id_59277.png
12363 ./max_images/shot5_546_id_59279.png
12364 ./max_images/shot55_682_id_619453.png
12365 ./max_images/shot55_759_id_619784.png
12366 ./max_images/shot55_759_id_619760.png
12367 ./max_images/shot55_759_id_619768.png
12368 ./max_images/shot55_759_id_619769.png
12369 ./max_images/shot55_759_id_619772.png
12370 ./max_images/shot55_991_id_621533.png
12371 ./max_images/shot56_1589_id_637395.png
12372 ./max_images/shot56_1801_id_638268.png
12373 ./max_images/shot56_1913_id_638743.png
12374 ./max_images/shot56_1915_id_638746.png
12375 ./max_images/shot56_1923_id_638779.png
12376 ./max_images/shot56_2288_id_64034

12542 ./max_images/shot62_1823_id_703818.png
12543 ./max_images/shot62_1879_id_704019.png
12544 ./max_images/shot62_368_id_697008.png
12545 ./max_images/shot62_368_id_697024.png
12546 ./max_images/shot62_368_id_697028.png
12547 ./max_images/shot62_368_id_697029.png
12548 ./max_images/shot62_368_id_697030.png
12549 ./max_images/shot62_371_id_697072.png
12550 ./max_images/shot62_608_id_698029.png
12551 ./max_images/shot62_613_id_698046.png
12552 ./max_images/shot62_613_id_698045.png
12553 ./max_images/shot62_615_id_698061.png
12554 ./max_images/shot62_822_id_699197.png
12555 ./max_images/shot62_904_id_699968.png
12556 ./max_images/shot62_904_id_699972.png
12557 ./max_images/shot62_904_id_699949.png
12558 ./max_images/shot62_907_id_700014.png
12559 ./max_images/shot62_908_id_700017.png
12560 ./max_images/shot62_908_id_700018.png
12561 ./max_images/shot62_909_id_700025.png
12562 ./max_images/shot63_1132_id_711306.png
12563 ./max_images/shot63_1132_id_711312.png
12564 ./max_images/shot63_15

12729 ./max_images/shot70_1280_id_789462.png
12730 ./max_images/shot70_1280_id_789463.png
12731 ./max_images/shot70_1282_id_789471.png
12732 ./max_images/shot70_1282_id_789470.png
12733 ./max_images/shot70_1284_id_789477.png
12734 ./max_images/shot70_1489_id_790482.png
12735 ./max_images/shot70_1761_id_791524.png
12736 ./max_images/shot70_192_id_783373.png
12737 ./max_images/shot70_2_id_782414.png
12738 ./max_images/shot70_208_id_783478.png
12739 ./max_images/shot70_210_id_783492.png
12740 ./max_images/shot70_210_id_783495.png
12741 ./max_images/shot70_334_id_784356.png
12742 ./max_images/shot70_492_id_785271.png
12743 ./max_images/shot70_854_id_787351.png
12744 ./max_images/shot70_854_id_787352.png
12745 ./max_images/shot70_858_id_787360.png
12746 ./max_images/shot70_858_id_787359.png
12747 ./max_images/shot70_860_id_787364.png
12748 ./max_images/shot70_860_id_787363.png
12749 ./max_images/shot70_862_id_787368.png
12750 ./max_images/shot70_862_id_787367.png
12751 ./max_images/shot70_8

12917 ./max_images/shot77_1091_id_871176.png
12918 ./max_images/shot77_1091_id_871181.png
12919 ./max_images/shot77_1101_id_871248.png
12920 ./max_images/shot77_1104_id_871266.png
12921 ./max_images/shot77_1557_id_873135.png
12922 ./max_images/shot77_1859_id_874596.png
12923 ./max_images/shot77_1906_id_875029.png
12924 ./max_images/shot77_1906_id_875039.png
12925 ./max_images/shot77_1906_id_875044.png
12926 ./max_images/shot77_1906_id_875011.png
12927 ./max_images/shot77_221_id_867228.png
12928 ./max_images/shot77_228_id_867261.png
12929 ./max_images/shot77_304_id_867636.png
12930 ./max_images/shot7_738_id_81808.png
12931 ./max_images/shot7_740_id_81814.png
12932 ./max_images/shot7_740_id_81816.png
12933 ./max_images/shot77_50_id_866464.png
12934 ./max_images/shot77_551_id_868875.png
12935 ./max_images/shot77_558_id_868918.png
12936 ./max_images/shot77_672_id_869332.png
12937 ./max_images/shot77_672_id_869337.png
12938 ./max_images/shot7_780_id_82011.png
12939 ./max_images/shot77_901_i

13104 ./max_images/shot89_1797_id_1000330.png
13105 ./max_images/shot89_1807_id_1000382.png
13106 ./max_images/shot89_1819_id_1000430.png
13107 ./max_images/shot89_1821_id_1000444.png
13108 ./max_images/shot89_1822_id_1000451.png
13109 ./max_images/shot89_1930_id_1000779.png
13110 ./max_images/shot89_519_id_993881.png
13111 ./max_images/shot89_519_id_993883.png
13112 ./max_images/shot89_519_id_993875.png
13113 ./max_images/shot89_519_id_993877.png
13114 ./max_images/shot89_523_id_993897.png
13115 ./max_images/shot89_523_id_993896.png
13116 ./max_images/shot89_613_id_994275.png
13117 ./max_images/shot89_613_id_994276.png
13118 ./max_images/shot89_919_id_996093.png
13119 ./max_images/shot89_946_id_996174.png
13120 ./max_images/shot89_947_id_996182.png
13121 ./max_images/shot89_951_id_996199.png
13122 ./max_images/shot89_957_id_996235.png
13123 ./max_images/shot90_125_id_1001795.png
13124 ./max_images/shot90_129_id_1001826.png
13125 ./max_images/shot90_129_id_1001825.png
13126 ./max_image